In [224]:
'''
This module contains all functions relating to the cleaning and exploration of structured data sets; mostly in pandas format
'''

##pip install category_encoders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders import *
from IPython.display import display
from collections import Counter
import scipy.stats as sp
import datetime as dt

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [196]:
def get_cat_feats(data=None):
    '''
    Returns the categorical features in a data set
    Parameters:
    -----------
        data: DataFrame or named Series 
    Returns:
    -------
        List
            A list of all the categorical features in a dataset.
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    cat_features = data.select_dtypes(include=['object']).columns

    return list(cat_features)

In [197]:
def get_num_feats(data=None):
    '''
    Returns the numerical features in a data set
    Parameters:
    -----------
        data: DataFrame or named Series 
    Returns:
    -------
        List:
            A list of all the numerical features in a dataset.
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    num_features = data.select_dtypes(exclude=['object', 'datetime64']).columns

    return list(num_features)

In [198]:
def get_unique_counts(data=None):
    '''
    Gets the unique count of categorical features in a data set.
    Parameters
    -----------
        data: DataFrame or named Series 
    Returns
    -------
        DataFrame or Series
            Unique value counts of the features in a dataset.
    
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    features = get_cat_feats(data)
    temp_len = []

    for feature in features:
        temp_len.append(len(data[feature].unique()))
        
    df = list(zip(features, temp_len))
    df = pd.DataFrame(df, columns=['Feature', 'Unique Count'])
    df = df.style.bar(subset=['Unique Count'], align='mid')
    return df

In [199]:
def display_missing(data=None, plot=False):
    '''
    Display missing values as a pandas dataframe.
    Parameters
    ----------
        data: DataFrame or named Series
        plot: bool, Default False
            Plots missing values in dataset as a heatmap
    
    Returns
    -------
        Matplotlib Figure:
            Heatmap plot of missing values
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    df = data.isna().sum()
    df = df.reset_index()
    df.columns = ['features', 'missing_counts']

    missing_percent = round((df['missing_counts'] / data.shape[0]) * 100, 1)
    df['missing_percent'] = missing_percent

    if plot:
        plot_missing(data)
        return df
    else:
        return df

In [200]:
def cat_summarizer(data, x=None, y=None, hue=None, palette='Set1', verbose=True):
    '''
    Helper function that gives a quick summary of a given column of categorical data
    Parameters:
    ---------------------------
        dataframe: pandas dataframe
        x: str.
            horizontal axis to plot the labels of categorical data, y would be the count.
        y: str. 
            vertical axis to plot the labels of categorical data, x would be the count.
        hue: str. i
            if you want to compare it another variable (usually the target variable)
        palette: array, list.
            Colour of the plot
    Returns:
    ----------------------
        Quick Stats of the data and also the count plot
    '''
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = data[column_interested]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.countplot(x=x, y=y, hue=hue, data=data, palette=palette)
    plt.show()

In [201]:
def _space():
    print('\n')
def _match_date(data):
    '''
        Return a list of columns that matches the DateTime expression
    '''
    mask = data.sample(20).astype(str).apply(lambda x : x.str.match(r'(\d{2,4}-\d{2}-\d{2,4})+').all())
    return set(data.loc[:, mask].columns)


def display_rows(data,num=2):
    '''
    Displays the required number of rows
    
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")

    return data.head(num)

In [202]:
def plot_missing(data=None):
    '''
    Plots the data as a heatmap to show missing values
    Parameters
    ----------
        data: DataFrame, array, or list of arrays.
            The data to plot.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    sns.heatmap(data.isnull(), cbar=True)
    plt.show()

In [203]:
def class_count(data=None, features=None, plot=False, save_fig=False):
    '''
    Displays the number of classes in a categorical feature.
    Parameters:
    
        data: Pandas DataFrame or Series
            Dataset for plotting.
        features: Scalar, array, or list. 
            The categorical features in the dataset, if None, 
            we try to infer the categorical columns from the dataframe.
        plot: bool, Default False.
            Plots the class counts as a barplot
        save_fig: bool, Default False.
            Saves the plot to the current working directory.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    if features is None:
        features = get_cat_feats(data)

                        

    for feature in features:
        if data[feature].nunique() > 15:
            print("Unique classes in {} too large".format(feature))
        else:
            print('Class Count for', feature)
            display(pd.DataFrame(data[feature].value_counts()))

    if plot:
        countplot(data, features, save_fig=save_fig)

In [204]:
def get_date_cols(data=None):
    '''
    Returns the Datetime columns in a data set.
    Parameters
    ----------
        data: DataFrame or named Series
            Data set to infer datetime columns from.
        convert: bool, Default True
            Converts the inferred date columns to pandas DateTime type
    Returns:
    -------
        List
         Date column names in the data set
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    #Get existing date columns in pandas Datetime64 format
    date_cols = set(data.dtypes[data.dtypes == 'datetime64[ns, UTC]'].index)
    #infer Date columns 
    date_cols = date_cols.union(_match_date(data))
       
    return date_cols

In [205]:
def bivariate_stats(data):
    cat_feats = get_cat_feats(data=data)
    counter = 1
    while counter<len(cat_feats):
        val1 = get_cat_feats(data=datainput)[counter-1]
        val2 = get_cat_feats(data=datainput)[counter]
        if (data[val1].nunique() > 15) or (data[val2].nunique() > 15):
            print('Number of unique values too large')
        else:
            freqtab = pd.crosstab(data[val1], data[val2])
            print("Frequency table")
            print("============================")
            print(freqtab)
            print("============================")
            chi2, pval, dof, expected = sp.chi2_contingency(freqtab)
            print("ChiSquare test statistic: ",chi2)
            print("p-value: ",pval)
            _space()
        counter= counter+1

In [206]:
def bivariate_stats_target(data, target):
    cat_feats = get_cat_feats(data=data)
    for i in cat_feats:
        if (data[i].nunique() > 20):
            print('Number of Unique values too large')
        else:
            freqtab = pd.crosstab(data[i], data[target])
            print("Frequency table")
            print("============================")
            print(freqtab)
            print("============================")
            chi2, pval, dof, expected = sp.chi2_contingency(freqtab)
            print("ChiSquare test statistic: ",chi2)
            print("p-value: ",pval)
            _space()

In [207]:
def describe(data=None, name='', date_cols=None, show_categories=False, plot_missing=False, target = None):
    '''
    Calculates statistics and information about a data set. Information displayed are
    shapes, size, number of categorical/numeric/date features, missing values,
    dtypes of objects etc.
    Parameters:
    --------------------
        data: Pandas DataFrame
            The data to describe.
        name: str, optional
            The name of the data set passed to the function.
        date_cols: list/series/array
            Date column names in the data set.
        show_categories: bool, default False
            Displays the unique classes and counts in each of the categorical feature in the data set.
        plot_missing: bool, default True
            Plots missing values as a heatmap
    Returns:
    -------
        None
    '''
    
    if data is None:
        raise ValueError("data: Expecting a DataFrame or Series, got 'None'")

    ## Get categorical features
    cat_features = get_cat_feats(data)
    
    #Get numerical features
    num_features = get_num_feats(data)

    print('First five data points')
    display(data.head())
    _space()

    print('Random five data points')
    display(data.sample(5))
    _space()

    print('Last five data points')
    display(data.tail())
    _space()

    print('Shape of {} data set: {}'.format(name, data.shape))
    _space()

    print('Size of {} data set: {}'.format(name, data.size))
    _space()

    print('Data Types')
    print("Note: All Non-numerical features are identified as objects in pandas")
    display(pd.DataFrame(data.dtypes, columns=['Data Type']))
    _space()
    
    date_cols = get_date_cols(data)
    if len(date_cols) is not 0:
        print("Column(s) {} should be in Datetime format. Use the [to_date] function to convert to Pandas Datetime format".format(date_cols))
        _space()

    print('Numerical Features in Data set')
    print(num_features)
    _space()

    print('Categorical Features in Data set')
    display(cat_features)
    _space()

    print('Statistical Description of Columns')
    display(data.describe())
    _space()
    
    print('Description of Categorical Features')
    if cat_features != None:
        display(data.describe(include=[np.object, pd.Categorical]).T)
        _space()
          
    print('Unique class Count of Categorical features')
    display(get_unique_counts(data))
    _space()

    if show_categories:     
        print('Classes in Categorical Columns')
        print("-"*30)
        class_count(data, cat_features)
        _space()

    print('Missing Values in Data')
    display(display_missing(data))
    _space()
  
    print('Pearson Correlation')
    print(data.corr())
    _space()
    
    print('Kendall Correlation')
    print(data.corr(method='kendall'))
    _space()
    
    print('Spearman Correlation')
    print(data.corr(method='spearman'))
    _space()
    
    print('Bivariant Stats between categorical features')
    print(bivariate_stats(data))
    _space()
    if target is not None:
        print('Bivariant Stats between cat feats and target variable')
        print(bivariate_stats_target(data, target))
        _space()
    print('')

In [228]:
def feature_processing(data=None, target=None, date_cols= None, percent = 70):
    data = drop_redundant(data)
    data = deal_with_missing_value(data, percent=70)
    if date_cols is not None:
        data_dates = data[date_cols]
        data_dates = date_processing(data=data_dates, date_cols = date_cols)
        X = data.drop(date_cols, axis=1)
    X = labelencoder(data = X)
    data = pd.concat([X, data_dates], axis=1)
    print('Pearson Correlation')
    print(data.corr()[target])
    _space()
    print('Kendall Correlation')
    print(data.corr(method='kendall')[target])
    _space()
    print('Spearman Correlation')
    print(data.corr(method='spearman')[target])
    return data

In [216]:
def date_processing(data=None, date_cols = None, utc=True):
    for i in date_cols:
        data[i] = pd.to_datetime(data[i], utc=utc)
        data[f'month_{i}'] = data[i].dt.month
        data[f'year_{i}'] = data[i].dt.year
        data[f'day_{i}'] = data[i].dt.day
        data[f'dayweek_{i}'] = data[i].dt.weekday
    return data

In [217]:
data.columns

Index(['product_id', 'account_id', 'geo_country', 'content_category',
       'content_message_local', 'content_message_global',
       'local_key_message_vault_id', 'global_key_message_vault_id', 'product',
       'segment_quant', 'segment_qual', 'touchpoint_channel_clm',
       'measure_source', 'call_id', 'calls_account_id', 'channel', 'is_clm',
       'key_message_id', 'id', 'call_date', 'fs_product_id', 'change_clm',
       'ISO3', 'ISO', 'areaAbbvie', 'geo_area', 'segment_concat',
       'content_message_concat', 'date', 'period_week', 'period_month',
       'month_date', 'year_date', 'day_date', 'dayweek_date',
       'month_period_week', 'year_period_week', 'day_period_week',
       'dayweek_period_week', 'month_period_month', 'year_period_month',
       'day_period_month', 'dayweek_period_month'],
      dtype='object')

In [218]:
data.corr()['product']

product_id                     0.023464
account_id                    -0.007413
geo_country                    0.104872
content_category              -0.060098
content_message_local          0.040966
content_message_global         0.094306
local_key_message_vault_id    -0.069982
global_key_message_vault_id    0.103402
product                        1.000000
segment_quant                  0.066813
segment_qual                   0.005764
touchpoint_channel_clm         0.131126
measure_source                -0.163023
call_id                        0.069786
calls_account_id              -0.083363
channel                        0.019319
is_clm                         0.146756
key_message_id                -0.047309
id                            -0.111656
call_date                     -0.068452
fs_product_id                  0.034225
change_clm                    -0.081659
ISO3                           0.055796
ISO                            0.055272
areaAbbvie                     0.032107


In [141]:
def drop_missing(data=None, percent=99):
    '''
    Drops missing columns with [percent] of missing data.
    Parameters:
    -------------------------
        data: Pandas DataFrame or Series.
        percent: float, Default 99
            Percentage of missing values to be in a column before it is eligible for removal.
    Returns:
    ------------------
        Pandas DataFrame or Series.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    
    missing_percent = (data.isna().sum() / data.shape[0]) * 100
    cols_2_drop = missing_percent[missing_percent.values >= percent].index
    print("Dropped {}".format(list(cols_2_drop)))
    #Drop missing values
    df = data.drop(cols_2_drop, axis=1)
    return df

def fill_missing_cats(data=None, cat_features=None, missing_encoding=None, missing_col=False):
    '''
    Fill missing values using the mode of the categorical features.
    Parameters:
    ------------------------
        data: DataFrame or name Series.
            Data set to perform operation on.
        cat_features: List, Series, Array.
            categorical features to perform operation on. If not provided, we automatically infer the categoricals from the dataset.
        missing_encoding: List, Series, Array.
            Values used in place of missing. Popular formats are [-1, -999, -99, '', ' ']
        missin_col: bool, Default True
            Creates a new column to capture the missing values. 1 if missing and 0 otherwise. This can sometimes help a machine learning model.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")

    if cat_features is None:
        cat_features = get_cat_feats(data)

    df = data.copy()
    #change all possible missing values to NaN
    if missing_encoding is None:
        missing_encoding = ['', ' ', -99, -999]

    df.replace(missing_encoding, np.NaN, inplace=True)
    
    for feat in cat_features:
        if missing_col:
            df[feat + '_missing_value'] = (df[feat].isna()).astype('int64')
        most_freq = df[feat].mode()[0]
        df[feat] = df[feat].replace(np.NaN, most_freq)
    
    return df

def fill_missing_num(data=None, num_features=None, method='mean', missing_col=False):
    '''
    fill missing values in numerical columns with specified [method] value
    Parameters:
        ------------------------------
        data: DataFrame or name Series.
            The data set to fill
        features: list.
            List of columns to fill
        method: str, Default 'mean'.
            method to use in calculating fill value.
        missing_col: bool, Default True
            Creates a new column to capture the missing values. 1 if missing and 0 otherwise. This can sometimes help a machine learning model.
    '''
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    
    if num_features is None:
        num_features = get_num_feats(data)
        #get numerical features with missing values
        temp_df = data[num_features].isna().sum()
        features = list(temp_df[num_features][temp_df[num_features] > 0].index)
        
    df = data.copy()
    for feat in features:
        if missing_col:
            df[feat + '_missing_value'] = (df[feat].isna()).astype('int64')
        if method is 'mean':
            mean = df[feat].mean()
            df[feat].fillna(mean, inplace=True)
        elif method is 'median':
            median = df[feat].median()
            df[feat].fillna(median, inplace=True)
        elif method is 'mode':
            mode = df[feat].mode()[0]
            df[feat].fillna(mode, inplace=True)
        else:
            raise ValueError("method: must specify a fill method, one of [mean, mode or median]'")
    return df

In [142]:
def deal_with_missing_value(data, percent=70):
    """
    this function automatically take care of missing values.
        It fills the missing values in categorical variables with mode of the particular column
        and fills the missing value numerical variables with mean of the particular column.
        It automatically drops columns with more than 70% missing values 
        """
    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    df1 = drop_missing(data=data, percent=percent)
    df2 = fill_missing_cats(data=df1)
    df = fill_missing_num(data=df2)
    return df

In [143]:
#Label Encoding for object to numeric conversion
def binaryencoder(data = None):
    """To avoid the curse of dimensionality, this function only encodes categorical features with less than 6 unique values"""
    features = get_cat_feats(data=data)
    cols = []
    for feature in features:
        if data[feature].nunique() < 4:
            cols.append(feature)
        
    enc = BinaryEncoder(cols=cols).fit(data)
    data = enc.transform(data)
    return data

def onehotencoder(data = None):
    """To avoid the curse of dimensionality, this function only encodes categorical features with less than 6 unique values"""
    features = get_cat_feats(data=data)
    cols = []
    for feature in features:
        if data[feature].nunique() < 4:
            cols.append(feature)
        
    enc = OneHotEncoder(cols=cols).fit(data)
    data = enc.transform(data)
    return data
def labelencoder(data = None):
    features = get_cat_feats(data=data)
    for feat in features:
        data[feat] = le.fit_transform(data[feat].astype(str))
    return data
def sumencoder(data = None):
    features = get_cat_feats(data=data)
    cols = []
    for feature in features:
        if data[feature].nunique() < 4:
            cols.append(feature)
    enc = SumEncoder(cols = cols).fit_transform(data)
    data = enc
    return data
def catboostencoder(data, target):
    '''Data inputs must not be string'''
    X = data.drop(target, axis=1)
    y = data[target]
    features = get_cat_feats(data=X)
    enc = CatBoostEncoder(cols=features).fit(X,y)
    data = enc.transform(X, y)
    return data
def encode_data(data, method='binary'):
    """
        encodes categorical variables automatically using binary encoding for columns with less than 4 unique values
        then label encode all other variables
        method takes either binary or onehot or sumencode. default is binary

        
    """
    if method == 'binary':
        data = binaryencoder(data=data)
    elif method== 'onehot':
        data = onehotencoder(data=data)
    elif method == 'sumencode':
        data = sumencode(data=data)
    data = labelencoder(data=data)
    return data
    

In [144]:
def drop_redundant(data):
    '''
    Removes features with the same value in all cell. Drops feature If Nan is the second unique class as well.
    Parameters:
    -----------------------------
        data: DataFrame or named series.
    
    Returns:
        DataFrame or named series.
    '''

    if data is None:
        raise ValueError("data: Expecting a DataFrame/ numpy2d array, got 'None'")
    
    #get columns
    cols_2_drop = _nan_in_class(data)
    print("Dropped {}".format(cols_2_drop))
    df = data.drop(cols_2_drop, axis=1)
    return df
def _nan_in_class(data):
    cols = []
    for col in data.columns:
        if len(data[col].unique()) == 1:
            cols.append(col)

        if len(data[col].unique()) == 2:
            if np.nan in list(data[col].unique()):
                cols.append(col)

    return cols

In [145]:
def get_qcut(data=None, col=None, q=None, duplicates='drop', return_type='float64'):
    '''
    Cuts a series into bins using the pandas qcut function
    and returns the resulting bins as a series for merging.
    Parameter:
    -------------
        data: DataFrame, named Series
            Data set to perform operation on.
        col: str
            column to cut/binnarize.
        q: integer or array of quantiles
            Number of quantiles. 10 for deciles, 4 for quartiles, etc. Alternately array of quantiles, e.g. [0, .25, .5, .75, 1.] for quartiles.
        duplicates: Default 'drop',
            If bin edges are not unique drop non-uniques.
        return_type: dtype, Default (float64)
            Dtype of series to return. One of [float64, str, int64]
    
    Returns:
    --------
        Series, 1D-Array
    '''

    temp_df = pd.qcut(data[col], q=q, duplicates=duplicates).to_frame().astype('str')
    #retrieve only the qcut categories
    df = temp_df[col].str.split(',').apply(lambda x: x[0][1:]).astype(return_type)
    
    return df

In [244]:
encode_data(data, method='binary')

,product_id_0,product_id_1,product_id_2,account_id,id_mail_template,id_approved_email,date,geo_country,email_activity_event_type,email_activity_sent_email,...,areaAbbvie_0,areaAbbvie_1,areaAbbvie_2,areaAbbvie_3,geo_area_0,geo_area_1,geo_area_2,geo_area_3,segment_concat,content_message_concat
0,0,0,1,678,21,4482,6952,37,5,4478,...,0,0,0,1,0,0,0,1,230,126
1,0,0,1,678,21,4482,6952,37,2,4478,...,0,0,0,1,0,0,0,1,230,126
2,0,0,1,678,21,8483,11303,37,5,8477,...,0,0,0,1,0,0,0,1,230,126
3,0,0,1,678,21,8483,11303,37,2,8477,...,0,0,0,1,0,0,0,1,230,126
4,0,0,1,935,43,6858,9370,7,2,6853,...,0,0,0,1,0,0,0,1,230,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168528,1,0,0,12195,119,14191,64,23,10,14121,...,0,1,0,1,0,1,0,1,0,68
168529,1,0,0,12195,119,14191,59,23,10,14121,...,0,1,0,1,0,1,0,1,0,0
168530,1,0,0,12195,119,14191,13288,23,10,14121,...,0,1,0,1,0,1,0,1,0,0
168531,1,0,0,12195,119,14191,65,23,10,14121,...,0,1,0,1,0,1,0,1,0,0


In [246]:
datainput = pd.read_csv(r"C:\Users\Ighdaro Emwinghare\Downloads\data2.csv")
datainput2 = pd.read_csv(r"C:\Users\Ighdaro Emwinghare\Downloads\data1.csv")

In [247]:
data = pd.concat([datainput, datainput2])

In [248]:
describe(data=data, name='', date_cols=['period_week','period_month', 'date'], show_categories=False, plot_missing=False, target='product')

First five data points


,product_id,account_id,id_mail_template,id_approved_email,date,geo_country,email_activity_event_type,email_activity_sent_email,content_category,content_message_local,...,call_date,fs_product_id,change_clm,con_size,ISO3,ISO,areaAbbvie,geo_area,segment_concat,content_message_concat
0,a001t00000JKjL4AAL,0011t00000IByzYAAT,a1n5J000000TYIKQA4,a1t5J000000U3kwQAC,2020-05-13T12:58:17.000Z,RUS,Opened_vod,a1t5J000000U3kwQAC,Safety,Улучшенный профиль польза/риск,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a001t00000JKjL4AAL,0011t00000IByzYAAT,a1n5J000000TYIKQA4,a1t5J000000U3kwQAC,2020-05-13T12:58:17.000Z,RUS,Delivered_vod,a1t5J000000U3kwQAC,Safety,Улучшенный профиль польза/риск,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a001t00000JKjL4AAL,0011t00000IByzYAAT,a1n5J000000TYIKQA4,a1t5J000000gRJfQAM,2020-06-04T14:09:23.000Z,RUS,Opened_vod,a1t5J000000gRJfQAM,Safety,Улучшенный профиль польза/риск,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a001t00000JKjL4AAL,0011t00000IByzYAAT,a1n5J000000TYIKQA4,a1t5J000000gRJfQAM,2020-06-04T14:09:23.000Z,RUS,Delivered_vod,a1t5J000000gRJfQAM,Safety,Улучшенный профиль польза/риск,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a001t00000JKjL4AAL,0011t00000N9Ig7AAF,a1n5J000000gCLyQAM,a1t5J000000gLaBQAU,2020-05-27T13:11:54.000Z,BRA,Delivered_vod,a1t5J000000gLaBQAU,Key Message Category does not apply,Local Key Message does not apply,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN




Random five data points


,product_id,account_id,id_mail_template,id_approved_email,date,geo_country,email_activity_event_type,email_activity_sent_email,content_category,content_message_local,...,call_date,fs_product_id,change_clm,con_size,ISO3,ISO,areaAbbvie,geo_area,segment_concat,content_message_concat
26050,a001t00000JKjL4AAL,0016F00001ladUfQAI,a1n5J000000TYKZQA4,a1t5J000000Tu4SQAS,2020-04-30T10:26:54.000Z,BRA,Opened_vod,a1t5J000000Tu4SQAS,Key Message Category does not apply,Local Key Message does not apply,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63694,a001v00002KP02YAAT,001G000001t4fElIAI,NaN,NaN,1/21/2020,Germany,NaN,NaN,Patient Support Programs,Besseres klinische Ansprechen durch AbbvieCare,...,1/21/2020,a001v00002KP02YAAT,CLM,True,DEU,DE,WE&C,WEC,N@(No segment),Patient Support Programs@AbbVie Care is a PSP ...
21070,a001v00002KP02YAAT,001G000001t52w2IAA,NaN,NaN,1/28/2020,Germany,NaN,NaN,Efficacy,RINVOQ+MTX erstmals signifikant hoehere Remiss...,...,1/28/2020,a001v00002KP02YAAT,CLM,True,DEU,DE,WE&C,WEC,B3@Rheum_Science First_Green,Efficacy@Remission rates vs placebo + MTX and ...
162552,a001v00002KP02YAAT,001G000001t4yTaIAI,NaN,NaN,3/10/2020,Germany,NaN,NaN,(No message),Verschiebe die Grenzen,...,3/10/2020,a001v00002KP02YAAT,CLM,True,DEU,DE,WE&C,WEC,R@(No segment),(No message)@(No message)@Verschiebe die Grenzen
36185,a001v00002KP02YAAT,001G000001t56E3IAI,NaN,NaN,2/7/2020,Germany,NaN,NaN,Efficacy,RINVOQ konsistent hohe Remissionsraten bei all...,...,2/7/2020,a001v00002KP02YAAT,CLM,True,DEU,DE,WE&C,WEC,C3@Rheum_Trust First_Yellow,"Efficacy@Consistent remission rates, & sig inh..."




Last five data points


...

,Feature,Unique Count
0,product_id,4
1,account_id,12196
2,id_mail_template,120
3,id_approved_email,14192
4,date,13398
5,geo_country,51
6,email_activity_event_type,11
7,email_activity_sent_email,14122
8,content_category,9
9,content_message_local,126




Missing Values in Data


,features,missing_counts,missing_percent
0,product_id,0,0.0
1,account_id,33160,14.6
2,id_mail_template,168533,74.3
3,id_approved_email,168533,74.3
4,date,165,0.1
5,geo_country,1,0.0
6,email_activity_event_type,168695,74.4
7,email_activity_sent_email,168695,74.4
8,content_category,84983,37.5
9,content_message_local,84980,37.5




Pearson Correlation
                  is_clm  activity_channel
is_clm               1.0               NaN
activity_channel     NaN               NaN


Kendall Correlation
                  is_clm  activity_channel
is_clm               1.0               NaN
activity_channel     NaN               NaN


Spearman Correlation
                  is_clm  activity_channel
is_clm               1.0               NaN
activity_channel     NaN               NaN


Bivariant Stats between categorical features
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Number of unique values too large
Frequency table
product                      ABC_Rinvoq_Rheum  Rinvoq_RA  Rinvoq_RA_KAM
glo

IndexError: list index out of range

In [235]:
feature_processing(data=data, target='product', date_cols=['period_week','period_month', 'date'], percent = 70)

Dropped ['indication', 'therapeutic_area', 'activity_channel', 'con_size']
Dropped ['id_mail_template', 'id_approved_email', 'email_activity_event_type', 'email_activity_sent_email']


C:\Users\Ighdaro Emwinghare\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Ighdaro Emwinghare\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Ighdaro Emwinghare\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a c

Pearson Correlation
product_id                     0.023464
account_id                    -0.007413
geo_country                    0.104872
content_category              -0.060098
content_message_local          0.040966
content_message_global         0.094306
local_key_message_vault_id    -0.069982
global_key_message_vault_id    0.103402
product                        1.000000
segment_quant                  0.066813
segment_qual                   0.005764
touchpoint_channel_clm         0.131126
measure_source                -0.163023
call_id                        0.069786
calls_account_id              -0.083363
channel                        0.019319
is_clm                         0.146756
key_message_id                -0.047309
id                            -0.111656
call_date                     -0.068452
fs_product_id                  0.034225
change_clm                    -0.081659
ISO3                           0.055796
ISO                            0.055272
areaAbbvie          

,product_id,account_id,geo_country,content_category,content_message_local,content_message_global,local_key_message_vault_id,global_key_message_vault_id,product,segment_quant,...,day_period_week,dayweek_period_week,month_period_month,year_period_month,day_period_month,dayweek_period_month,month_date,year_date,day_date,dayweek_date
0,0,678,37,7,124,15,152,3,1,8,...,11,0,5,2020,1,4,5,2020,13,2
1,0,678,37,7,124,15,152,3,1,8,...,11,0,5,2020,1,4,5,2020,13,2
2,0,678,37,7,124,15,152,3,1,8,...,1,0,6,2020,1,0,6,2020,4,3
3,0,678,37,7,124,15,152,3,1,8,...,1,0,6,2020,1,0,6,2020,4,3
4,0,935,7,4,58,9,91,5,1,8,...,25,0,5,2020,1,4,5,2020,27,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168528,3,6878,23,2,73,12,9,0,0,8,...,24,0,2,2020,1,5,2,2020,25,1
168529,3,6878,23,2,73,12,9,5,0,8,...,17,0,2,2020,1,5,2,2020,19,2
168530,3,6878,23,2,73,12,9,5,0,8,...,23,0,3,2020,1,6,3,2020,24,1
168531,3,6878,23,2,73,12,9,5,0,8,...,24,0,2,2020,1,5,2,2020,26,2
